In [1]:
import cv2
import numpy as np
import os
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
import json #подключили библиотеку для работы с json
from pprint import pprint #подключили Pprint для красоты выдачи текста


import descriptors_getters


def plot_gray(image):
    %matplotlib qt
    plt.figure(figsize=(20,15))
    return plt.imshow(image, cmap='Greys_r')

def image_scale(image, scale_to_height):
    scale_factor = scale_to_height / image.shape[0]
    width  = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    image = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
    return image

def preprocess(image, scale_to_height =100):
    image = image_scale(image, scale_to_height)
    #image = cv2.medianBlur(image, 3)
    #image = cv2.GaussianBlur(image, (3, 3), 0)
    #_, image = cv2.threshold(image,0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #image = canny_threshold(image)
    return image

In [2]:
data = pd.read_csv("aktau_vs_all.csv")
k_s = range(50, 500, 25)
max_lr_iterations_s = range(3,25,2)
C_s = np.logspace(-2,2,7)
scale_factor_s = np.arange(1,2.5,0.2)
nfeatures_s = range(30, 520, 20)
nlevels_s = [4,6,8,10]
edgeThreshold_s = range(10,45,5)
firstlevel_s = [0,1,2]
WTA_K_s = [1,2,3]
patchSize_s = range(10, 45, 5)
fastThreshold_s = range(10, 35, 5)

In [3]:
cols = ['k', 'max_lr_iterations', 'C', 'scale_factor', 'nfeatures', 'nlevels', 'edgeThreshold',
        'firstLevel', 'WTA_K', 'patchSize', 'fastThreshold', 'precision_score', 'recall_score', 'f1_score', 'accuracy_score']
results =pd.DataFrame(columns=cols)

In [4]:
def pipeline(data, k, max_lr_iterations, C, scale_factor, nfeatures, nlevels, edgeThreshold, firstLevel,
             WTA_K, patchSize, fastThreshold):
    
    def get_descriptors(img):   
        orb = cv2.ORB_create(nfeatures = nfeatures, #500
                            scaleFactor = scale_factor,#1.2
                            nlevels = nlevels,
                            edgeThreshold = edgeThreshold,
                            firstLevel = firstLevel,
                            WTA_K = WTA_K,
                            patchSize = patchSize,
                            fastThreshold = fastThreshold)
        kp = orb.detect(img,None)
        kp, des = orb.compute(img, kp)

        return kp, des
    
    y = data['Class']
    X = data.drop(['Class'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30) #random_state=42
    # Обучение кластеризатора
    dico = []

    for path in X_train['Path']:
        img = cv2.imread(path, 0)
        img = preprocess(img)
        if img.shape[1] > 1000 or img.shape[1]< 30:
            continue
        kp, des = get_descriptors(img)
        if des is not None:
            for d in des:
                dico.append(d)
        else:
            assert(False)

    #batch_size = np.size(image_names) * 3
    batch_size = 64
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=0).fit(dico)#random_state=42
    kmeans.verbose = False

    # Обучение классификатора
    histo_list = []

    for path in X_train['Path']:
        img = cv2.imread(path,0)
        img = preprocess(img)
        # find the keypoints with STAR
        # compute the descriptors with BRIEF
        if img.shape[1] > 1000 or img.shape[1]< 30:
            histo = np.zeros(k)
            histo_list.append(histo)
            continue
        kp, des = get_descriptors(img)
        histo = np.zeros(k)
        nkp = np.size(kp)    
        if des is not None:
            for d in des:

                idx = kmeans.predict([d])
                histo[idx] += 1/nkp 

        histo_list.append(histo)
        
    X = np.array(histo_list)
    Y = y_train
    lr = LogisticRegression(C= C, class_weight='balanced', max_iter=max_lr_iterations, n_jobs = -1)
    lr.fit(X, Y)
    
    
    # Предсказание классификатором
    y_pred = []
    for path in X_test['Path']:
        img = cv2.imread(path,0)
        img = preprocess(img, scale_factor)
        if img.shape[1] > 1000 or img.shape[1]< 30:
            x = np.zeros(k)
            res = lr.predict([x])
            y_pred.append(res)
            continue
        #print(img.shape)   
        kp, des = get_descriptors(img)    
        x = np.zeros(k)
        nkp = np.size(kp)

        if des is not None:
            for d in des:
                idx = kmeans.predict([d])
                x[idx] += 1/nkp
        res = lr.predict([x])
        #for e in res[0]:
            #row.append(e)
        y_pred.append(res)
    precision_score = metrics.precision_score(y_test.values, y_pred)
    recall_score = metrics.recall_score(y_test.values, y_pred)
    accuracy_score = metrics.accuracy_score(y_test.values, y_pred)
    f1_score = metrics.f1_score(y_test.values, y_pred)
    return precision_score, recall_score, f1_score, accuracy_score

In [ ]:
itertn = 0
for k in k_s:
    for max_lr_iterations in max_lr_iterations_s:
        for C in C_s:
                for scale_factor in scale_factor_s:
                    for nfeatures in nfeatures_s:
                        for nlevels in nlevels_s:
                            for edgeThreshold in edgeThreshold_s:
                                for firstlevel in firstlevel_s:
                                    for WTA_K in WTA_K_s:
                                        for patchSize in patchSize_s:
                                            for fastThreshold in fastThreshold_s:
                                                
                                                try:
                                                    precision_score, recall_score, f1_score, accuracy_score = pipeline(data, k, max_lr_iterations, C, scale_factor,
                                                    nfeatures, nlevels, edgeThreshold, firstlevel, WTA_K, patchSize, fastThreshold)
                                                    raw = [k, max_lr_iterations, C, scale_factor, nfeatures, nlevels, edgeThreshold, firstlevel,
                                                          WTA_K, patchSize, fastThreshold, precision_score, recall_score, f1_score, accuracy_score]
                                                    rawdf = pd.DataFrame([raw], columns = cols)
                                                    results = results.append(rawdf, ignore_index=True)
                                                    results.to_csv('results.csv')
                                                except Exception as e:
                                                    #print(e)
                                                    pass

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

...

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM